In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
df = pd.read_csv("Telco_customer_churn.csv")
df.head()

In [ ]:
df.shape


In [ ]:
df.columns

In [ ]:
df.drop(["Country","State","City","Zip Code","Latitude","Longitude"], axis=1, inplace=True)

In [ ]:
df.drop("Count", axis=1, inplace=True)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df["Churn Value"].value_counts()

In [ ]:
df.drop(["Churn Label","Churn Score","Churn Reason"], axis=1, inplace=True)

In [ ]:
df.shape

In [ ]:
X = df.drop("Churn Value", axis=1)
y = df["Churn Value"]

X.head()

In [ ]:
X_encoded = pd.get_dummies(X, drop_first=True)

X_encoded.head()

In [ ]:
X_encoded.shape

In [ ]:
X = df.drop(["Churn Value", "CustomerID"], axis=1)

In [ ]:
X_encoded = pd.get_dummies(X, drop_first=True)
X_encoded.shape

In [ ]:
X.nunique().sort_values(ascending=False).head(10)

In [ ]:
X.dtypes

In [ ]:
X["Total Charges"] = pd.to_numeric(X["Total Charges"], errors="coerce")
X["Monthly Charges"] = pd.to_numeric(X["Monthly Charges"], errors="coerce")
X["CLTV"] = pd.to_numeric(X["CLTV"], errors="coerce")
X["Lat Long"] = pd.to_numeric(X["Lat Long"], errors="coerce")

In [ ]:
X.dtypes

In [ ]:
X_encoded = pd.get_dummies(X, drop_first=True)
X_encoded.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, 
    y, 
    test_size=0.2, 
    random_state=42
)

In [ ]:
X_encoded.isnull().sum().sum()

In [ ]:
X_encoded = X_encoded.fillna(0)

In [ ]:
# Remove unwanted columns again safely
X = df.drop(["Churn Value", "CustomerID", "CLTV"], axis=1)
y = df["Churn Value"]

In [ ]:
X["Total Charges"] = pd.to_numeric(X["Total Charges"], errors="coerce")
X["Monthly Charges"] = pd.to_numeric(X["Monthly Charges"], errors="coerce")
X["Lat Long"] = pd.to_numeric(X["Lat Long"], errors="coerce")

In [ ]:
X = X.fillna(0)

In [ ]:
X_encoded = pd.get_dummies(X, drop_first=True)

In [ ]:
X_encoded.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded,
    y,
    test_size=0.2,
    random_state=42
)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = LogisticRegression(max_iter=2000)

model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
feature_importance = pd.DataFrame({
    "Feature": X_encoded.columns,
    "Coefficient": model.coef_[0]
})

feature_importance = feature_importance.sort_values(
    by="Coefficient", 
    ascending=False
)

feature_importance.head(10)

Key Business Insights

*Customers with month-to-month contracts show higher churn probability.

*Higher monthly charges are associated with increased churn.

*Customers with lower tenure months are more likely to churn.

*Lack of tech support increases churn risk.

*Long-term contract customers show lower churn probability.

Conclusion

The Logistic Regression model achieved an accuracy of 80%.
The model effectively predicts customer churn and identifies major factors influencing churn behavior.

Key drivers include contract type, tenure, and monthly charges.
Business strategies such as promoting long-term contracts and improving support services can reduce churn.

In [ ]:
# Get churn probability scores
churn_prob = model.predict_proba(X_test_scaled)[:, 1]

# Create dataframe for segmentation
segmentation_df = X_test.copy()
segmentation_df["Churn Probability"] = churn_prob
segmentation_df["Actual Churn"] = y_test.values

# Create segments
segmentation_df["Customer Segment"] = pd.cut(
    segmentation_df["Churn Probability"],
    bins=[0, 0.3, 0.7, 1],
    labels=["Loyal", "Dormant", "At Risk"]
)

segmentation_df["Customer Segment"].value_counts()

In [ ]:
!pip install shap

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Model
model = LogisticRegression(max_iter=2000)
model.fit(X_train_scaled, y_train)

In [ ]:
import shap

explainer = shap.LinearExplainer(model, X_train_scaled)
shap_values = explainer.shap_values(X_test_scaled)

shap.summary_plot(
    shap_values,
    X_test_scaled,
    feature_names=X_encoded.columns
)

In [ ]:
# Get churn probabilities
y_prob = model.predict_proba(X_test_scaled)[:, 1]

# Create dataframe for segmentation
segmentation_df = X_test.copy()
segmentation_df["Churn_Probability"] = y_prob
segmentation_df["Actual_Churn"] = y_test.values

segmentation_df.head()

In [ ]:
def assign_segment(row):
    if row["Churn_Probability"] > 0.6:
        return "At Risk"
    elif row["Tenure Months"] > 24:
        return "Loyal"
    else:
        return "Dormant"

segmentation_df["Customer_Segment"] = segmentation_df.apply(assign_segment, axis=1)

segmentation_df["Customer_Segment"].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data=segmentation_df, x="Customer_Segment")
plt.title("Customer Segmentation Based on Churn Risk")
plt.show()

In [ ]:
segmentation_df.to_csv("Customer_Segmentation_Output.csv", index=False)

In [ ]:
import sqlite3

# Create in-memory database
conn = sqlite3.connect(":memory:")

# Store dataframe as SQL table
df.to_sql("customers", conn, index=False, if_exists="replace")

print("Table created successfully!")

In [ ]:
query1 = """
SELECT Contract,
       COUNT(*) AS Total_Customers,
       SUM("Churn Value") AS Total_Churned
FROM customers
GROUP BY Contract
"""

pd.read_sql(query1, conn)

In [ ]:
query2 = """
SELECT "Churn Value",
       AVG("Monthly Charges") AS Avg_Monthly_Charges
FROM customers
GROUP BY "Churn Value"
"""

pd.read_sql(query2, conn)

In [ ]:
query3 = """
SELECT "Internet Service",
       COUNT(*) AS Total_Customers,
       SUM("Churn Value") AS Churned_Customers
FROM customers
GROUP BY "Internet Service"
"""

pd.read_sql(query3, conn)

In [ ]:
query4 = """
SELECT CustomerID,
       "Total Charges"
FROM customers
ORDER BY "Total Charges" DESC
LIMIT 10
"""

pd.read_sql(query4, conn)